In [1]:
import torch
import torch.nn.functional as F

In [132]:
not None

True

In [131]:
p = torch.rand(512, 256)/256
pz = torch.zeros(512, 256)

# p = torch.rand(1, 2)
# pz = torch.zeros(1, 2)

projection_4 = torch.concat(
    (torch.concat((p, pz, pz, pz), dim=0),
    torch.concat((pz, p, pz, pz), dim=0),
    torch.concat((pz, pz, p, pz), dim=0),
    torch.concat((pz, pz, pz, p), dim=0),)
, dim=1)
# projection_4 = torch.concat((projection_4, projection_4, projection_4, projection_4), dim=1)
print(projection_4.shape)

w = torch.rand(2048, 512)

torch.Size([2048, 1024])


In [127]:
w.T.shape

torch.Size([512, 2048])

In [128]:
projection_4.T.shape

torch.Size([1024, 2048])

In [129]:
projection_4

tensor([[0.0005, 0.0019, 0.0003,  ..., 0.0000, 0.0000, 0.0000],
        [0.0029, 0.0013, 0.0034,  ..., 0.0000, 0.0000, 0.0000],
        [0.0026, 0.0007, 0.0024,  ..., 0.0000, 0.0000, 0.0000],
        ...,
        [0.0000, 0.0000, 0.0000,  ..., 0.0038, 0.0022, 0.0007],
        [0.0000, 0.0000, 0.0000,  ..., 0.0012, 0.0002, 0.0038],
        [0.0000, 0.0000, 0.0000,  ..., 0.0018, 0.0027, 0.0002]])

In [130]:
(w.T@projection_4.T)

RuntimeError: mat1 and mat2 shapes cannot be multiplied (512x2048 and 1024x2048)

In [13]:


torch.manual_seed(27)

# Example tensor of shape (1, 1, H, W)
DIM = 4
W = torch.rand(DIM, DIM)
P1 = torch.rand(int(DIM/2), DIM)
P2 = torch.rand(DIM, int(DIM/2))



In [16]:
P1.shape

torch.Size([2, 4])

In [17]:
W.shape

torch.Size([4, 4])

In [15]:
P1@W#@P2

tensor([[1.5872, 1.1626, 1.6426, 0.9742],
        [0.4341, 0.6854, 0.8121, 0.5155]])

In [73]:
i = torch.rand([100, 256])
p1 = torch.rand([512, 256]) #P torch.Size([256, 512])
w = torch.rand([1536, 512]) #W torch.Size([256, 1536])
p2 = torch.rand([768, 1536]) #P torch.Size([256, 768])

In [74]:
(i@p1.T@w.T@p2.T).shape

torch.Size([100, 768])

In [5]:
DIM1 = 12
DIM2 = 4
W = torch.rand(DIM1, DIM2)
P1 = torch.rand(int(DIM1/2), DIM1)
P2 = torch.rand(DIM2, int(DIM2/2))

In [6]:
r = P1@W@P2
print(W.shape)
print(r.shape)
r

torch.Size([12, 4])
torch.Size([6, 2])


tensor([[5.6443, 5.0776],
        [6.5575, 5.7987],
        [7.7880, 7.0714],
        [7.1346, 6.4522],
        [6.2099, 5.7591],
        [7.6796, 6.9450]])

In [28]:
DIM1 = 50257
DIM2 = 512
W = torch.rand(DIM1, DIM2)
P2 = torch.rand(DIM2, int(DIM2/2))

In [29]:
(W@P2).shape

torch.Size([502, 2])

In [31]:
P2

tensor([[0.6864, 0.3059],
        [0.6783, 0.0089],
        [0.8684, 0.8320],
        [0.6172, 0.5390],
        [0.8979, 0.8956]])

In [7]:
# xb = 512
# yb = 2048
# xs = 256
# ys = 1024
# W = torch.rand(xb, yb)
# P1 = torch.rand(xs, xb)
# P2 = torch.rand(yb, ys)

# r = P1@W@P2
# print(W.shape)
# print(r.shape)
# r

In [9]:
from lizrd.core.misc import Linear


l = Linear(
    4, #xs
    8, #xb
    init_type = "kaiming_uniform",
    init_scale = 1.0
)
l

Linear(in_features=4, out_features=8, bias=False)

In [22]:
# from lizrd.train.train_utils import get_model
# from research.conditional.utils.model_utils import get_ff_layer
# from research.projected_distillation.llm import ProjectedFeedForward
# dmodel = 256
# dff = 1024
# projected_dmodel = dmodel*2
# projected_dff = dff*2
# n_blocks = 8
# init_type = "kaiming_uniform"
# init_scale= 1.0
# block_modules = {
#     "FF": lambda: ProjectedFeedForward(
#             dmodel, dff, projected_dmodel, projected_dff, init_type=init_type, init_scale= init_scale
#         ),
#     "Attention": 
# }

# model = get_model(
#         max_length=256,
#         vocab_size=50048,
#         block_modules=block_modules,
#         dm=dmodel,
#         n_blocks=n_blocks,
#         device=(
#             torch.device("cuda")
#         ),  # in case of  DDP/FSDP, we initialize the model on CPU and move it to the GPU later
#         init_type=init_type,
#         init_scale=init_scale,
#         ddp_enabled=False,
#         fsdp_enabled=False,
#         fsdp_param_precision=False,
#         fsdp_mixed_precision_ignore_classes=False,
#         fsdp_offload_params=False,
#         fsdp_min_num_params=False,
#         fsdp_modules_to_wrap=False,
#         activation_checkpointing_modules=False,
#     )

SyntaxError: expression expected after dictionary key and ':' (902681333.py, line 15)